In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import time, sys
from IPython.display import clear_output

def show(img, *masks):
    global width
    global height
    try:
        if(img.shape[2] == 1):
            plt.imshow(img.reshape(width, height), vmin=0, vmax=1)
        else:
            plt.imshow(img, vmin=0, vmax=1)
    except:
        plt.imshow(img, vmin=0, vmax=1)
    for mask in masks:
        plt.imshow(mask.reshape(width, height), alpha=0.4, vmin=0, vmax=1)
    plt.show()

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [4]:
import os
import cv2
import imageio
import numpy as np
from PIL import Image
from skimage import color, io
from sklearn.model_selection import train_test_split

mdirectory = "training/data/resized_masks"
idirectory = "training/data/resized_images"

length = len(os.listdir(mdirectory))
width = 224
height = 224

masks = []
images = []

for number, path in enumerate(os.listdir(mdirectory)):
    _, threshed = cv2.threshold(cv2.imread(os.path.join(mdirectory, path), 0), 127, 255, cv2.THRESH_BINARY)
    masks.append(threshed.reshape(224, 224, 1).astype("float32") / 255)
    images.append(cv2.imread(os.path.join(idirectory, path)).astype("float32") / 255)
    update_progress(number / length)
X_train, X_test, y_train, y_test = train_test_split(np.array(images), np.array(masks), test_size=0.2)

Progress: [####################] 99.9%


In [7]:
from scipy import ndimage
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt

def find_center(scores):
    highest_score = list(scores.values())[0]
    comp_score = 0
    num_added = 0
    for key in scores:
        if scores[key] > 0.95 * highest_score:
            comp_score += key
            num_added += 1
    if(num_added > 0):
        return comp_score/num_added
    else:
        return 0.5

def find_path(mask):
    num_horizontal_windows = 14
    horizontal_window_size = int(width / num_horizontal_windows)
    
    num_vertical_windows = 2
    vertical_window_size = int(height / num_vertical_windows)
    
    differences = []
    for vertical_num in range(num_vertical_windows):
        scores = {}
        for horizontal_num in range(num_horizontal_windows):
            current_slice = mask[vertical_num * vertical_window_size:(vertical_num + 1) * vertical_window_size, 
                                 horizontal_num * horizontal_window_size:(horizontal_num + 1) * horizontal_window_size]
            scores[horizontal_num] = np.average(current_slice)
        scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        differences.append(find_center(scores))
    return differences[0] - differences[1]

directions = []
for number, mask in enumerate(y_train):
    mask = mask.reshape(width, height)
    directions.append(find_path(mask))
#     break
    update_progress(number / len(y_train))
directions = minmax_scale(np.asarray(directions).astype('float32'), feature_range=(0, 1), axis=0, copy=True)

Progress: [####################] 99.9%


In [ ]:
#split video

import cv2
import os
from PIL import Image

root = "./training/"
cap = cv2.VideoCapture(root + "videos/iphone1.mov")
file_count = len(os.listdir(root + "images"))
skip_count = 0
skip_factor = 10
while(cap.isOpened()):
    ret, frame = cap.read()
    if(skip_count % skip_factor == 0):
        # final = cv2.resize(cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE), (320, 240))
        cv2.imwrite(root + "images/frame%d.png" % file_count, frame)
        file_count += 1
    skip_count += 1
    printProgressBar(skip_count, cap.get(7) / skip_factor)

In [ ]:
#convert annotations

import os
import imageio
import json
import numpy as np
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt

polygons = "./training/polygons/"
masks = "./training/masks/"
count = 0
for r, d, f, in os.walk(polygons):
    for name in f:
        with open(polygons + name) as points:
            parsed = json.load(points)
            img = Image.new('L', (1080, 1920), 0)
            if(len(parsed["shapes"]) > 0):
                for shape in parsed["shapes"]:
                    polygon = list(map(tuple, shape["points"]))
                    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
                mask = np.array(img)
            plt.imsave(masks + name.replace("json", "png"), mask, cmap=plt.cm.gray)
            count += 1
            update_progress(count/len(f))

In [ ]:
#resize images

import cv2
import os

img_dir = "./training/images/"
mask_dir = "./training/masks/"

img_output = "./training/data/resized_images/"
mask_output = "./training/data/resized_masks/"

for n, filename in enumerate(os.listdir(mask_dir)):
    img = cv2.resize(cv2.imread(img_dir + filename), (224, 224), interpolation=cv2.INTER_AREA)
    mask = cv2.resize(cv2.imread(mask_dir + filename), (224, 224), interpolation=cv2.INTER_AREA)
    plt.imsave(img_output + filename, img, cmap=plt.cm.gray)
    plt.imsave(mask_output + filename, mask, cmap=plt.cm.gray)
    update_progress(n/len(os.listdir(mask_dir)))

In [ ]:
#rename

import os
import json
def new_name(name):
    return ("frame" + str(int(name[5:-4])).zfill(4) + ".png")

dir = r"C:\\projects\\blindpilot\training\\images\\"
for filename in os.listdir(dir):
    os.rename(dir+filename, dir + new_name(filename))
    # with open(dir + filename, "r") as jsonFile:
    #     data = json.load(jsonFile)
    # data["imagePath"] = data["imagePath"].replace("jpg", "png")

    # with open(dir + filename, "w") as jsonFile:
    #     json.dump(data, jsonFile)